<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #4: Add new clips to a Zooniverse workflow</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Dec 8th, 2021</h5>

# Set up and requirements

### Import Python packages

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t4_utils as t4
#import utils.server_utils as server_utils

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

Packages loaded successfully


### Choose your project

In [30]:
project = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'), value='Koster_Sea…

### Initiate sql and zoo project

In [31]:
# Initiate db
db_info_dict = t_utils.initiate_db(project.value)

Enter your username for SNIC server········
Enter your password for SNIC server········
Updated sites
Updated movies
Updated species


In [32]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project.value)

Enter your Zooniverse user········
Enter your Zooniverse password········


# Retrieve info about zooniverse clips

In [33]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(project_name = project.value, 
                                                    db_info_dict = db_info_dict,
                                                    zoo_project = zoo_project,
                                                    zoo_info = ["subjects"])

Retrieving subjects from Zooniverse
subjects were retrieved successfully
Updated subjects
The database has a total of 2342 frame subjects and 7362 clip subjects have been updated


# Retrieve info about movies hosted in the server

In [34]:
# Store info about the movies available in the server
available_movies_df = t4.retrieve_movie_info_from_server(project_name = project.value,
                                                         db_info_dict = db_info_dict)

### Select the movie you want to upload to Zooniverse

In [35]:
movie_i = t4.movie_to_upload(available_movies_df = available_movies_df)

Dropdown(description='Movie to upload:', layout=Layout(width='50%'), options=('000114 TMBL-ROV 2000 Säckenrev…

### Check if movie is already in Zooniverse

Remember to query the newest zooniverse data to get the most up to date list of clips uploaded

In [36]:
# Check movie hasn't been uploaded to Zooniverse
t4.check_movie_uploaded(movie_i = movie_i.value,
                        db_info_dict = db_info_dict)

010424 Säckenrevet beta Tape 74.mov has not been uploaded to Zooniverse yet


### Specify the number of clips and clip length

In [37]:
clip_selection = t4.select_clip_n_len(movie_i = movie_i.value,
                                      db_info_dict = db_info_dict)

interactive(children=(Dropdown(description='Length of clips:', options=(10, 5), style=DescriptionStyle(descrip…

### Review the clips that will be created

In [38]:
t4.review_clip_selection(clip_selection = clip_selection, 
                         movie_i = movie_i.value)

You are about to create 2 clips from 010424 Säckenrevet beta Tape 74.mov
starting at 0:00:00 and ending at 0:00:28


### Create the clips

In [40]:
clips_to_upload_df = t4.create_clips(available_movies_df = available_movies_df, 
                                     movie_i = movie_i.value,
                                     db_info_dict = db_info_dict,
                                     clip_selection = clip_selection,  
                                     project_name = project.value)

0.00b [00:00, ?b/s]

/cephyr/NOBACKUP/groups/snic2021-6-9/koster_movies/01724008_1.mov


 50%|#####     | 118M/237M [00:31<00:31, 3.73Mb/s] 


SFTPError: Garbage packet received

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

### Review clips

In [ ]:
t4.check_clip_size(clip_paths = clips_to_upload_df["clip_path"].unique())

### Video modifications

In [ ]:
# Work in progress
clip_modification = t4.select_modification()

In [ ]:
# Modify the clips
clips_ready_df = t4.modify_clips(clips_to_upload_df = clips_to_upload_df.reset_index(drop=True), 
                                 movie_i = movie_i.value,
                                 clip_modification = clip_modification.label, 
                                 modification_details = clip_modification.value)

In [ ]:
# Review the size of the modified clips
t4.check_clip_size(clip_paths = clips_ready_df["modif_clip_path"].unique())

In [41]:
# Compare the original and modified clips
t4.compare_clips(df = clips_ready_df)

Dropdown(description='Select original clip:', layout=Layout(width='50%'), options=('0 No movie', 'Film_3.mp4_c…

Output()

### Set Zooniverse metadata

In [ ]:
upload_to_zoo, sitename, created_on = t4.set_zoo_metadata(db_info_dict = db_info_dict, 
                                                          df = clips_ready_df,
                                                          project_name = project.value)

### Upload clips to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [ ]:
t4.upload_clips_to_zooniverse(upload_to_zoo = upload_to_zoo, 
                              sitename = sitename,
                              created_on = created_on,
                              project = str(zoo_info_dict["subjects"]["project_id"].unique()[0]))

In [ ]:
#END